In [1]:
!pip install --user pyarrow

You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
from xml.etree.ElementTree import iterparse
import pandas as pd
from tqdm import tqdm

In [3]:
def xml_gen():
    with open('/projects/bpms/pstjohn/uniref100.xml') as source:
        context = iterparse(source, events=('start', 'end'))
        event, root = next(context)

        for i, (event, elem) in enumerate(context):
            if elem.tag == '{http://uniprot.org/uniref}entry' and event == 'start':
                curr_id = elem.get('id')

            elif elem.tag == '{http://uniprot.org/uniref}entry' and event == 'end':
                root.clear()

            elif elem.tag == '{http://uniprot.org/uniref}representativeMember' and event == 'end':
                dbref, seq = elem.getchildren()
                props = {p.get('type'): p.get('value') for p in dbref.getchildren()}
                props['UniRef100 ID'] = curr_id               
                props['Sequence'] = seq.text

                yield props
#                 if curr_id == 'UniRef100_UPI00129EFDE7':
#                     break

from itertools import islice

def grouper(iterable, n):
    iterable = iter(iterable)
    while True:
        tup = tuple(islice(iterable, n))
        if tup:
            yield tup
        else:
            break

In [ ]:
for i, data in tqdm(enumerate(grouper(xml_gen(), int(1E5)))):
    df = pd.DataFrame(data)
    
    df['length'] = df['length'].astype(int)
    df = df[['UniRef100 ID', 'UniRef90 ID', 'UniRef50 ID', 'NCBI taxonomy', 'length', 'Sequence']]
    
    df.to_parquet(f'/scratch/pstjohn/uniparc/uniref100/{i:03d}.parquet')

295it [27:25,  5.33s/it]

In [ ]:
df.head()